In [1]:
import pandas as pd

df = pd.read_csv('tickets.csv', encoding='utf-8')

In [2]:
df.head()

,customer_name,customer_email,product_name,issue_description,opening_date,criticality
0,Marcos Vinícius Oliveira,suporte@edutech.solutions,CourseGen Studio,Cliente Marcos Vinícius Oliveira da EduTech So...,2025-04-15,Alta
1,Fernanda Oliveira Schmidt,fernanda.schmidt@eductech.io,CourseGen Studio,Identificamos falhas intermitentes no módulo d...,2025-05-10,Alta
2,Lucas Henrique Silva,lucas.silva@bancoconsulta.com.br,RiskGen,Identificamos um problema crítico relacionado ...,2025-04-15,Crítica
3,Lucas Oliveira Santos,lucas.santos@saudetech.com.br,ClinicaGPT,O sistema apresenta falhas críticas na criptog...,2025-04-15,Crítica
4,Mariana Oliveira Silva,mariana.silva@educaonline.com.br,EduMentor AI,Detalhamento dos problemas encontrados com a p...,2025-04-15,Alta


In [4]:
df["product_name"].value_counts()

product_name
ClinicaGPT          238
CourseGen Studio    231
EduMentor AI        231
RiskGen             225
FinBrain            220
Name: count, dtype: int64

## Análise de Tickets Abertos por Dia

In [ ]:
# !pip install plotly

  Using cached plotly-6.5.2-py3-none-any.whl.metadata (8.5 kB)
  Using cached narwhals-2.17.0-py3-none-any.whl.metadata (14 kB)
Using cached plotly-6.5.2-py3-none-any.whl (9.9 MB)
Using cached narwhals-2.17.0-py3-none-any.whl (444 kB)


In [6]:
import plotly.express as px

df['opening_date'] = pd.to_datetime(df['opening_date'])

tickets_por_dia = df.groupby(df['opening_date'].dt.date).size().reset_index(name='quantidade')

fig = px.line(
    tickets_por_dia,
    x='opening_date',
    y='quantidade',
    title='Quantidade de Tickets Abertos por Dia',
    labels={'opening_date': 'Data', 'quantidade': 'Quantidade de Tickets'},
    markers=True,
    template='plotly_white'
)

fig.update_layout(
    xaxis_title='Data de Abertura',
    yaxis_title='Quantidade de Tickets',
    hovermode='x unified',
    height=500
)

fig.show()

## Análise de Tickets Abertos por Mês

In [7]:
tickets_por_mes = df.groupby(df['opening_date'].dt.to_period('M')).size().reset_index(name='quantidade')
tickets_por_mes['opening_date'] = tickets_por_mes['opening_date'].astype(str)

fig_mes = px.bar(
    tickets_por_mes,
    x='opening_date',
    y='quantidade',
    title='Quantidade de Tickets Abertos por Mês',
    labels={'opening_date': 'Mês', 'quantidade': 'Quantidade de Tickets'},
    template='plotly_white',
    text='quantidade'
)

fig_mes.update_traces(textposition='outside')

fig_mes.update_layout(
    xaxis_title='Mês',
    yaxis_title='Quantidade de Tickets',
    hovermode='x unified',
    height=500
)

fig_mes.show()

In [13]:
df

,customer_name,customer_email,product_name,issue_description,opening_date,criticality
0,Marcos Vinícius Oliveira,suporte@edutech.solutions,CourseGen Studio,Cliente Marcos Vinícius Oliveira da EduTech So...,2025-04-15,Alta
1,Fernanda Oliveira Schmidt,fernanda.schmidt@eductech.io,CourseGen Studio,Identificamos falhas intermitentes no módulo d...,2025-05-10,Alta
2,Lucas Henrique Silva,lucas.silva@bancoconsulta.com.br,RiskGen,Identificamos um problema crítico relacionado ...,2025-04-15,Crítica
3,Lucas Oliveira Santos,lucas.santos@saudetech.com.br,ClinicaGPT,O sistema apresenta falhas críticas na criptog...,2025-04-15,Crítica
4,Mariana Oliveira Silva,mariana.silva@educaonline.com.br,EduMentor AI,Detalhamento dos problemas encontrados com a p...,2025-04-15,Alta
...,...,...,...,...,...,...
1140,Ana Beatriz Schmidt,ana.schmidt@edutech.com.br,EduMentor AI,Problema específico detectado: erro na criptog...,2025-03-15,Crítica
1141,Mariana Oliveira Schmidt,mariana.schmidt@educabrain.com.br,CourseGen Studio,O cliente EducaBrain reporta falhas intermiten...,2025-03-18,Alta
1142,Larissa Almeida Cavalcanti,larissa.almeida@healthcarepro.com.br,ClinicaGPT,Dificuldade grave no controle de acesso via RB...,2025-03-10,Crítica
1143,Carlos Eduardo Schmidt,carlos.schmidt@bancofinanceiro.com.br,FinBrain,A API de integração com ERPs financeiros apres...,2025-04-10,Alta


## Distribuição de Tickets por Produto ao Longo do Ano

In [ ]:
from typing import Union, Optional, List
import plotly.graph_objects as go

def plot_distribuicao_produtos_ano(
    dataframe: pd.DataFrame,
    tipo_visualizacao: str = "linhas",
    periodo: str = "M",
    produtos: Optional[List[str]] = None,
    mostrar_estatisticas: bool = False
) -> go.Figure:  # sourcery skip: extract-method
    """Gera análise interativa da distribuição de tickets por produto ao longo do tempo.
    
    Agrupa os dados por período temporal (dia/semana/mês/trimestre/ano) e produto,
    exibindo a evolução com múltiplas opções de visualização e análises.
    
    Args:
        dataframe: DataFrame contendo os dados de tickets
        tipo_visualizacao: Tipo de gráfico ('linhas', 'barras', 'area' ou 'scatter')
        periodo: Período de agregação ('D'=dia, 'W'=semana, 'M'=mês, 'Q'=trimestre, 'Y'=ano)
        produtos: Lista opcional de produtos a exibir (None = todos)
        mostrar_estatisticas: Se True, exibe estatísticas descritivas no console
    
    Returns:
        Figura Plotly com o gráfico gerado e análises
        
    Raises:
        ValueError: Se tipo_visualizacao ou periodo inválidos
    """
    tipos_validos = {'linhas', 'barras', 'area', 'scatter'}
    periodos_validos = {'D', 'W', 'M', 'Q', 'Y'}
    
    if tipo_visualizacao not in tipos_validos:
        raise ValueError(f"tipo_visualizacao deve ser um de {tipos_validos}")
    
    if periodo not in periodos_validos:
        raise ValueError(f"periodo deve ser um de {periodos_validos}")
    
    dados = dataframe.copy()
    
    if produtos:
        dados = dados[dados['product_name'].isin(produtos)]
        if dados.empty:
            raise ValueError(f"Nenhum dado encontrado para os produtos: {produtos}")
    
    distribuicao = dados.groupby(
        [dados['opening_date'].dt.to_period(periodo), 'product_name']
    ).size().reset_index(name='quantidade')
    
    distribuicao['opening_date'] = distribuicao['opening_date'].astype(str)
    distribuicao = distribuicao.sort_values(['opening_date', 'product_name'])
    
    mapa_periodos = {'D': 'Dia', 'W': 'Semana', 'M': 'Mês', 'Q': 'Trimestre', 'Y': 'Ano'}
    periodo_label = mapa_periodos[periodo]
    
    mapa_cores = {
        'linhas': 'lines+markers',
        'area': 'lines',
        'scatter': 'markers'
    }
    
    if tipo_visualizacao == 'linhas':
        fig = px.line(
            distribuicao,
            x='opening_date',
            y='quantidade',
            color='product_name',
            title=f'Distribuição de Tickets por Produto ({periodo_label})',
            labels={'opening_date': periodo_label, 'quantidade': 'Tickets', 'product_name': 'Produto'},
            markers=True,
            template='plotly_white'
        )
    
    elif tipo_visualizacao == 'barras':
        fig = px.bar(
            distribuicao,
            x='opening_date',
            y='quantidade',
            color='product_name',
            title=f'Distribuição de Tickets por Produto ({periodo_label})',
            labels={'opening_date': periodo_label, 'quantidade': 'Tickets', 'product_name': 'Produto'},
            barmode='group',
            template='plotly_white'
        )
    
    elif tipo_visualizacao == 'area':
        fig = px.area(
            distribuicao,
            x='opening_date',
            y='quantidade',
            color='product_name',
            title=f'Distribuição de Tickets por Produto ({periodo_label})',
            labels={'opening_date': periodo_label, 'quantidade': 'Tickets', 'product_name': 'Produto'},
            template='plotly_white'
        )
    
    else:  # scatter
        fig = px.scatter(
            distribuicao,
            x='opening_date',
            y='quantidade',
            color='product_name',
            size='quantidade',
            title=f'Distribuição de Tickets por Produto ({periodo_label})',
            labels={'opening_date': periodo_label, 'quantidade': 'Tickets', 'product_name': 'Produto'},
            template='plotly_white'
        )
    
    fig.update_layout(
        xaxis_title=periodo_label,
        yaxis_title='Quantidade de Tickets',
        hovermode='x unified',
        height=600,
        legend=dict(
            yanchor='top',
            y=0.99,
            xanchor='right',
            x=0.99,
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='gray',
            borderwidth=1
        ),
        font=dict(size=11),
        plot_bgcolor='rgba(240, 240, 240, 0.5)'
    )
    
    if mostrar_estatisticas:
        print(f"\n{'='*60}")
        print(f"ESTATÍSTICAS - Distribuição de Tickets ({periodo_label})")
        print(f"{'='*60}")
        print(f"\nTotal de tickets: {distribuicao['quantidade'].sum()}")
        print(f"Período: {distribuicao['opening_date'].min()} a {distribuicao['opening_date'].max()}")
        print(f"\nPor Produto:")
        stats = distribuicao.groupby('product_name')['quantidade'].agg(['sum', 'mean', 'min', 'max'])
        stats.columns = ['Total', 'Média', 'Mínimo', 'Máximo']
        print(stats.to_string())
        print(f"\n{'='*60}\n")
    
    fig.show()
    return fig

fig_distribuicao = plot_distribuicao_produtos_ano(df, mostrar_estatisticas=True)


ESTATÍSTICAS - Distribuição de Tickets (Mês)

Total de tickets: 1145
Período: 2025-01 a 2025-08

Por Produto:
                  Total      Média  Mínimo  Máximo
product_name                                      
ClinicaGPT          238  59.500000       3     135
CourseGen Studio    231  38.500000       1     141
EduMentor AI        231  33.000000       1     140
FinBrain            220  36.666667       2     133
RiskGen             225  56.250000       2     135




In [12]:
import random
from datetime import datetime, timedelta


def generate_random_dates(start_date: str, end_date: str, count: int = 1) -> list:
    """Generate random dates within a specified date range.
    
    Creates a list of random dates between the start_date and end_date,
    both provided in YYYY-MM-DD format. Useful for generating test data
    or simulating date distributions.
    
    Args:
        start_date: Start date in YYYY-MM-DD format
        end_date: End date in YYYY-MM-DD format
        count: Number of random dates to generate (default: 1)
    
    Returns:
        List of randomly generated date strings in YYYY-MM-DD format
        
    Raises:
        ValueError: If date format is invalid or start_date is after end_date
    """
    
    # Validate and parse dates
    try:
        date_start = datetime.strptime(start_date, '%Y-%m-%d')
        date_end = datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError as error:
        raise ValueError(f"Date format must be YYYY-MM-DD. Error: {error}") from error

    # Validate date range
    if date_start > date_end:
        raise ValueError(f"start_date ({start_date}) cannot be after end_date ({end_date})")

    # Calculate the difference in days
    date_diff = (date_end - date_start).days

    if date_diff < 0:
        raise ValueError("Invalid date range")

    # Generate random dates
    random_dates = []
    for _ in range(count):
        random_days = random.randint(0, date_diff)
        random_date = date_start + timedelta(days=random_days)
        random_dates.append(random_date.strftime('%Y-%m-%d'))

    return random_dates


# Test the function with examples
print("\n" + "="*70)
print("RANDOM DATE GENERATOR - USAGE EXAMPLES")
print("="*70)

# Example 1: Generate 5 random dates in 2025
example_1 = generate_random_dates('2025-01-01', '2025-12-31', count=5)
print(f"\n1. Random dates in 2025 (5 dates):")
for date in example_1:
    print(f"   • {date}")

# Example 2: Generate 10 random dates in a specific range
example_2 = generate_random_dates('2024-03-15', '2024-09-30', count=10)
print(f"\n2. Random dates between 2024-03-15 and 2024-09-30 (10 dates):")
for date in example_2:
    print(f"   • {date}")

# Example 3: Single date
example_3 = generate_random_dates('2023-01-01', '2023-01-31', count=1)
print(f"\n3. Single random date in January 2023:")
print(f"   • {example_3[0]}")

print("\n" + "="*70 + "\n")



RANDOM DATE GENERATOR - USAGE EXAMPLES

1. Random dates in 2025 (5 dates):
   • 2025-06-01
   • 2025-01-11
   • 2025-10-03
   • 2025-01-02
   • 2025-06-21

2. Random dates between 2024-03-15 and 2024-09-30 (10 dates):
   • 2024-09-20
   • 2024-04-18
   • 2024-07-27
   • 2024-05-22
   • 2024-05-10
   • 2024-04-23
   • 2024-07-12
   • 2024-05-04
   • 2024-09-05
   • 2024-05-11

3. Single random date in January 2023:
   • 2023-01-13




In [20]:

def reassign_product_dates_by_percentage(
    dataframe: pd.DataFrame,
    product_name: str,
    percentage: float,
    start_date: str,
    end_date: str
) -> pd.DataFrame:
    """Randomly reassign opening dates for a percentage of product records.
    
    This function selects a random sample of records for a specific product
    and reassigns their opening_date to new random dates within the specified
    date range. Useful for testing, data augmentation, or temporal distribution
    adjustments.
    
    Args:
        dataframe: Source DataFrame with 'opening_date' and 'product_name' columns
        product_name: Name of the product to modify
        percentage: Percentage of product records to reassign (0-100)
        start_date: Start date for new dates in YYYY-MM-DD format
        end_date: End date for new dates in YYYY-MM-DD format
    
    Returns:
        Modified DataFrame with reassigned dates for selected records
        
    Raises:
        ValueError: If product not found, invalid percentage, or date format issues
    """
    
    # Input validation
    if not (0 <= percentage <= 100):
        raise ValueError(f"Percentage must be between 0 and 100, got {percentage}")

    if product_name not in dataframe['product_name'].unique():
        available_products = dataframe['product_name'].unique().tolist()
        raise ValueError(
            f"Product '{product_name}' not found. Available products: {available_products}"
        )

    # Validate date format
    try:
        datetime.strptime(start_date, '%Y-%m-%d')
        datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError as error:
        raise ValueError(f"Date format must be YYYY-MM-DD. Error: {error}") from error

    # Create working copy
    result_df = dataframe.copy()

    # Filter records by product
    product_mask = result_df['product_name'] == product_name
    product_indices = result_df[product_mask].index.tolist()

    if not product_indices:
        raise ValueError(f"No records found for product '{product_name}'")

    # Calculate number of records to reassign
    num_to_reassign = max(1, int(len(product_indices) * (percentage / 100)))

    # Randomly select indices to reassign
    selected_indices = random.sample(product_indices, num_to_reassign)

    # Generate new random dates
    new_dates = generate_random_dates(start_date, end_date, count=num_to_reassign)

    # Update the DataFrame with new dates
    for idx, new_date in zip(selected_indices, new_dates):
        result_df.at[idx, 'opening_date'] = new_date

    # Convert opening_date back to datetime for consistency
    result_df['opening_date'] = pd.to_datetime(result_df['opening_date'])

    return result_df

In [21]:
df = pd.read_csv('tickets.csv', encoding='utf-8')

In [61]:
df_modified = reassign_product_dates_by_percentage(
    dataframe=df_modified,
    product_name="CourseGen Studio",
    percentage=70,
    start_date='2025-08-01',
    end_date='2025-09-30'
)


In [62]:
fig_plot = plot_distribuicao_produtos_ano(df_modified, mostrar_estatisticas=False)

In [63]:
df_modified.to_csv("tickets_modified.csv", index=False, encoding='utf-8')

In [67]:
df_test = pd.read_csv("tickets_modified.csv", encoding='utf-8')
df_test['opening_date'] = pd.to_datetime(df_test['opening_date'])

In [68]:
fig_distribuicao = plot_distribuicao_produtos_ano(df_test, mostrar_estatisticas=True)


ESTATÍSTICAS - Distribuição de Tickets (Mês)

Total de tickets: 1145
Período: 2025-01 a 2025-12

Por Produto:
                  Total      Média  Mínimo  Máximo
product_name                                      
ClinicaGPT          238  21.636364       3      63
CourseGen Studio    231  19.250000       2      86
EduMentor AI        231  23.100000       1     137
FinBrain            220  20.000000       1      64
RiskGen             225  18.750000      13      26


